In [1]:
#from google.cloud import bigquery
import pandas as pd

In [2]:
projectid = "ebmdatalab"

In [3]:
#update "euctr.euctr_2019_02" to most recent results table for a given month

euctr_country_data = pd.read_gbq('''
CREATE TEMPORARY FUNCTION
jsonArrayToSqlArray(inputArray STRING)
  RETURNS ARRAY <STRING>
  LANGUAGE js AS """
try {
  parts = JSON.parse(inputArray);
  if (Array.isArray(parts)) {
   return parts.map(x => JSON.stringify(x));
  } else if (parts instanceof Object) {
   return [JSON.stringify(parts)];
  }
} catch (e) {
  return "error"
}
""";

with
sponsor_info as(
select
eudract_number, eudract_number_with_country, sponsors
from (SELECT
  eudract_number, eudract_number_with_country, jsonArrayToSqlArray(sponsors) as sponsors
FROM
  `euctr.euctr_2019_02_01`)
left join unnest(sponsors) sponsors WITH OFFSET pos)

SELECT
eudract_number,
eudract_number_with_country,
regexp_extract(eudract_number_with_country, '-([^-]+)$') as trial_locations,
trim(json_extract(sponsors, "$.country"),'"')  as sponsor_country
from
sponsor_info
order by eudract_number
''', projectid, dialect = 'standard')

In [5]:
euctr_country_data.head()

,eudract_number,eudract_number_with_country,trial_locations,sponsor_country
0,2004-000007-18,2004-000007-18-SE,SE,Sweden
1,2004-000012-13,2004-000012-13-EE,EE,United States
2,2004-000012-13,2004-000012-13-CZ,CZ,United States
3,2004-000012-13,2004-000012-13-IT,IT,Italy
4,2004-000015-25,2004-000015-25-SK,SK,United States


In [6]:
trial_location_all = euctr_country_data[['eudract_number', 'trial_locations']].copy()

In [7]:
trial_location_dummies = pd.get_dummies(trial_location_all, columns=['trial_locations'])

In [8]:
trial_location_grouped = trial_location_dummies.groupby('eudract_number', as_index=False).max()

In [9]:
trial_location_grouped.head()

,eudract_number,trial_locations_3rd,trial_locations_AT,trial_locations_BE,trial_locations_BG,trial_locations_CZ,trial_locations_DE,trial_locations_DK,trial_locations_EE,trial_locations_ES,...,trial_locations_LV,trial_locations_MT,trial_locations_NL,trial_locations_NO,trial_locations_PL,trial_locations_PT,trial_locations_RO,trial_locations_SE,trial_locations_SI,trial_locations_SK
0,2004-000007-18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2004-000012-13,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2004-000015-25,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,2004-000016-10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2004-000020-32,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
trial_location_totals = trial_location_grouped.drop('eudract_number', axis=1).sum()

In [11]:
print(trial_location_totals)

trial_locations_3rd     1210
trial_locations_AT      3791
trial_locations_BE      5301
trial_locations_BG      1647
trial_locations_CZ      3890
trial_locations_DE     10280
trial_locations_DK      3551
trial_locations_EE       954
trial_locations_ES      8537
trial_locations_FI      2332
trial_locations_FR      4553
trial_locations_GB      8598
trial_locations_GR      1532
trial_locations_HR       276
trial_locations_HU      3955
trial_locations_IE      1059
trial_locations_IS       129
trial_locations_IT      7058
trial_locations_LT      1148
trial_locations_LU         7
trial_locations_LV       991
trial_locations_MT        18
trial_locations_NL      4878
trial_locations_NO       508
trial_locations_PL      2582
trial_locations_PT      1366
trial_locations_RO       226
trial_locations_SE      3602
trial_locations_SI       344
trial_locations_SK      1633
dtype: int64


In [14]:
sponsor_country_all = euctr_country_data[['eudract_number', 'sponsor_country']].copy()

In [15]:
sponsor_country_dummies = pd.get_dummies(sponsor_country_all, columns=['sponsor_country'])

In [16]:
sponsor_country_grouped = sponsor_country_dummies.groupby('eudract_number', as_index=False).max()

In [17]:
sponsor_country_totals = sponsor_country_grouped.drop('eudract_number', axis=1).sum()

In [18]:
print(sponsor_country_totals)

sponsor_country_Argentina                                  1
sponsor_country_Australia                                 97
sponsor_country_Austria                                 1277
sponsor_country_Belgium                                 2129
sponsor_country_Belize                                     3
sponsor_country_Benin                                      1
sponsor_country_Bermuda                                    3
sponsor_country_Bhutan                                     2
sponsor_country_Brazil                                     3
sponsor_country_Bulgaria                                  59
sponsor_country_Canada                                   127
sponsor_country_China                                      9
sponsor_country_Croatia                                   38
sponsor_country_Cyprus                                     4
sponsor_country_Czech Republic                           393
sponsor_country_Denmark                                 1889
sponsor_country_Djibouti

In [19]:
euctr_results_data = pd.read_gbq("""
select
trial_id as eudract_number,
max(results_expected) as results_expected,
max(has_results) as has_results
from `euctr.feb19_trials_csv` 
group by trial_id
""", projectid, dialect = 'standard')

In [20]:
country_and_results = pd.merge(trial_location_grouped, euctr_results_data, on='eudract_number')

In [21]:
country_and_results.head()

,eudract_number,trial_locations_3rd,trial_locations_AT,trial_locations_BE,trial_locations_BG,trial_locations_CZ,trial_locations_DE,trial_locations_DK,trial_locations_EE,trial_locations_ES,...,trial_locations_NL,trial_locations_NO,trial_locations_PL,trial_locations_PT,trial_locations_RO,trial_locations_SE,trial_locations_SI,trial_locations_SK,results_expected,has_results
0,2004-000007-18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,2004-000012-13,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,2004-000015-25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,2004-000016-10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2004-000020-32,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [22]:
countries = country_and_results.columns[1:31].values.tolist()
print(countries)

['trial_locations_3rd', 'trial_locations_AT', 'trial_locations_BE', 'trial_locations_BG', 'trial_locations_CZ', 'trial_locations_DE', 'trial_locations_DK', 'trial_locations_EE', 'trial_locations_ES', 'trial_locations_FI', 'trial_locations_FR', 'trial_locations_GB', 'trial_locations_GR', 'trial_locations_HR', 'trial_locations_HU', 'trial_locations_IE', 'trial_locations_IS', 'trial_locations_IT', 'trial_locations_LT', 'trial_locations_LU', 'trial_locations_LV', 'trial_locations_MT', 'trial_locations_NL', 'trial_locations_NO', 'trial_locations_PL', 'trial_locations_PT', 'trial_locations_RO', 'trial_locations_SE', 'trial_locations_SI', 'trial_locations_SK']


In [23]:
def country_reported(country):
    results_expected = 0
    expected_with_results = 0
    for row in country_and_results.index:
        if country_and_results.at[row, country] == 1 and country_and_results.at[row, 'results_expected'] == 1:
            results_expected += 1
        if country_and_results.at[row,country] == 1 and country_and_results.at[row, 'results_expected'] == 1 and country_and_results.at[row, 'has_results'] == 1:
            expected_with_results += 1
    return results_expected, expected_with_results
    

In [24]:
print(country_reported('trial_locations_PL'))

(627, 500)


In [25]:
location_country_reported_dict = {}
for country in countries:
    results_tuple = country_reported(country)
    if results_tuple[0] == 0:
        percentage = 0
    else:    
        percentage = round((results_tuple[1] / results_tuple[0]) * 100,2)
    location_country_reported_dict[country] = [results_tuple[0], results_tuple[1], percentage]

In [26]:
print(location_country_reported_dict)

{'trial_locations_3rd': [0, 0, 0], 'trial_locations_AT': [1035, 541, 52.27], 'trial_locations_BE': [889, 674, 75.82], 'trial_locations_BG': [481, 378, 78.59], 'trial_locations_CZ': [986, 790, 80.12], 'trial_locations_DE': [3323, 2148, 64.64], 'trial_locations_DK': [984, 466, 47.36], 'trial_locations_EE': [298, 240, 80.54], 'trial_locations_ES': [992, 803, 80.95], 'trial_locations_FI': [487, 352, 72.28], 'trial_locations_FR': [286, 175, 61.19], 'trial_locations_GB': [1731, 1163, 67.19], 'trial_locations_GR': [255, 192, 75.29], 'trial_locations_HR': [40, 33, 82.5], 'trial_locations_HU': [998, 791, 79.26], 'trial_locations_IE': [139, 96, 69.06], 'trial_locations_IS': [45, 20, 44.44], 'trial_locations_IT': [1179, 892, 75.66], 'trial_locations_LT': [357, 267, 74.79], 'trial_locations_LU': [1, 1, 100.0], 'trial_locations_LV': [307, 235, 76.55], 'trial_locations_MT': [2, 2, 100.0], 'trial_locations_NL': [466, 384, 82.4], 'trial_locations_NO': [30, 24, 80.0], 'trial_locations_PL': [627, 500, 7

In [27]:
reporting_by_country = pd.DataFrame.from_dict(location_country_reported_dict, orient='index',columns = ["Results Expected", "Results Available", "Percent Reported"])

In [28]:
print(reporting_by_country.sort_values(by = "Percent Reported", ascending = False))

                     Results Expected  Results Available  Percent Reported
trial_locations_MT                  2                  2            100.00
trial_locations_LU                  1                  1            100.00
trial_locations_RO                 33                 29             87.88
trial_locations_SI                 65                 54             83.08
trial_locations_SK                453                374             82.56
trial_locations_HR                 40                 33             82.50
trial_locations_NL                466                384             82.40
trial_locations_PT                195                158             81.03
trial_locations_ES                992                803             80.95
trial_locations_EE                298                240             80.54
trial_locations_CZ                986                790             80.12
trial_locations_NO                 30                 24             80.00
trial_locations_PL       

In [29]:
sponsor_country_and_results = pd.merge(sponsor_country_grouped, euctr_results_data, on='eudract_number')

In [30]:
sponsor_country_and_results.head()

,eudract_number,sponsor_country_Argentina,sponsor_country_Australia,sponsor_country_Austria,sponsor_country_Belgium,sponsor_country_Belize,sponsor_country_Benin,sponsor_country_Bermuda,sponsor_country_Bhutan,sponsor_country_Brazil,...,sponsor_country_Taiwan,sponsor_country_Turkey,sponsor_country_Ukraine,sponsor_country_United Arab Emirates,sponsor_country_United Kingdom,sponsor_country_United States,sponsor_country_United States Minor Outlying Islands,sponsor_country_Uruguay,results_expected,has_results
0,2004-000007-18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2004-000012-13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,2004-000015-25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,2004-000016-10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2004-000020-32,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [31]:
sponsor_countries = sponsor_country_and_results.columns[1:80].values.tolist()
print(sponsor_countries)

['sponsor_country_Argentina', 'sponsor_country_Australia', 'sponsor_country_Austria', 'sponsor_country_Belgium', 'sponsor_country_Belize', 'sponsor_country_Benin', 'sponsor_country_Bermuda', 'sponsor_country_Bhutan', 'sponsor_country_Brazil', 'sponsor_country_Bulgaria', 'sponsor_country_Canada', 'sponsor_country_China', 'sponsor_country_Croatia', 'sponsor_country_Cyprus', 'sponsor_country_Czech Republic', 'sponsor_country_Denmark', 'sponsor_country_Djibouti', 'sponsor_country_Estonia', 'sponsor_country_European Union', 'sponsor_country_Falkland Islands (Malvinas)', 'sponsor_country_Finland', 'sponsor_country_France', 'sponsor_country_France, Metropolitan', 'sponsor_country_French Guiana', 'sponsor_country_Georgia', 'sponsor_country_Germany', 'sponsor_country_Greece', 'sponsor_country_Greenland', 'sponsor_country_Hong Kong', 'sponsor_country_Hungary', 'sponsor_country_Iceland', 'sponsor_country_India', 'sponsor_country_Ireland', 'sponsor_country_Israel', 'sponsor_country_Italy', 'sponso

In [32]:
def sponsor_country_reported(country):
    results_expected = 0
    expected_with_results = 0
    for row in sponsor_country_and_results.index:
        if sponsor_country_and_results.at[row, country] == 1 and sponsor_country_and_results.at[row, 'results_expected'] == 1:
            results_expected += 1
        if sponsor_country_and_results.at[row,country] == 1 and sponsor_country_and_results.at[row, 'results_expected'] == 1 and sponsor_country_and_results.at[row, 'has_results'] == 1:
            expected_with_results += 1
    return results_expected, expected_with_results

In [33]:
sponsor_country_reported_dict = {}
for country in sponsor_countries:
    results_tuple = sponsor_country_reported(country)
    if results_tuple[0] == 0:
        percentage = 0
    else:    
        percentage = round((results_tuple[1] / results_tuple[0]) * 100,2)
    sponsor_country_reported_dict[country] = [results_tuple[0], results_tuple[1], percentage]

In [34]:
print(sponsor_country_reported_dict)

{'sponsor_country_Argentina': [1, 0, 0.0], 'sponsor_country_Australia': [28, 11, 39.29], 'sponsor_country_Austria': [509, 137, 26.92], 'sponsor_country_Belgium': [372, 280, 75.27], 'sponsor_country_Belize': [1, 1, 100.0], 'sponsor_country_Benin': [0, 0, 0], 'sponsor_country_Bermuda': [2, 0, 0.0], 'sponsor_country_Bhutan': [0, 0, 0], 'sponsor_country_Brazil': [0, 0, 0], 'sponsor_country_Bulgaria': [16, 10, 62.5], 'sponsor_country_Canada': [21, 2, 9.52], 'sponsor_country_China': [1, 0, 0.0], 'sponsor_country_Croatia': [5, 2, 40.0], 'sponsor_country_Cyprus': [3, 0, 0.0], 'sponsor_country_Czech Republic': [95, 58, 61.05], 'sponsor_country_Denmark': [690, 231, 33.48], 'sponsor_country_Djibouti': [0, 0, 0], 'sponsor_country_Estonia': [29, 20, 68.97], 'sponsor_country_European Union': [2, 2, 100.0], 'sponsor_country_Falkland Islands (Malvinas)': [0, 0, 0], 'sponsor_country_Finland': [156, 63, 40.38], 'sponsor_country_France': [432, 270, 62.5], 'sponsor_country_France, Metropolitan': [10, 7, 7

In [35]:
reporting_by_sponsor_country = pd.DataFrame.from_dict(sponsor_country_reported_dict, orient='index',columns = ["Results Expected", "Results Available", "Percent Reported"])

In [36]:
print(reporting_by_sponsor_country.sort_values(by = "Percent Reported", ascending = False))

                                                    Results Expected  \
sponsor_country_Singapore                                          1   
sponsor_country_European Union                                     2   
sponsor_country_Belize                                             1   
sponsor_country_Russian Federation                                 4   
sponsor_country_Netherlands Antilles                               1   
sponsor_country_Ukraine                                            1   
sponsor_country_Luxembourg                                         1   
sponsor_country_Jersey                                             1   
sponsor_country_Jamaica                                            1   
sponsor_country_Switzerland                                      643   
sponsor_country_Slovakia                                          19   
sponsor_country_Spain                                            416   
sponsor_country_Belgium                                         

[79 rows x 3 columns]


In [ ]:
reporting_by_country.to_csv('reporting_by_country.csv')

In [ ]:
reporting_by_sponsor_country.to_csv('reporting_by_sponsor_country.csv')